In [1]:
import os
import librosa
import jams
import muda

input_root = 'clipped_data'
output_root = os.path.join('augmented_data', 'drc')

# Initialize the DRC augmenter once (outside the loop)
drc = muda.deformers.DynamicRangeCompression(preset="music standard")

for subdir, _, files in os.walk(input_root):
    for file in files:
        if file.endswith('.wav'):
            input_path = os.path.join(subdir, file)
            rel_path = os.path.relpath(subdir, input_root)
            output_dir = os.path.join(output_root, rel_path)
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, file)

            # === Load audio ===
            y, sr = librosa.load(input_path, sr=None)

            # Create an empty JAMS object (needed by muda)
            jam = jams.JAMS()
            jam.file_metadata.duration = librosa.get_duration(y=y, sr=sr)
            jam = muda.jam_pack(jam, _audio=dict(y=y, sr=sr))

            # === Apply DRC ===
            for jam_out in drc.transform(jam):
                muda.save(output_path, output_path.replace(".wav", ".jams", jam_out))

None
None


CalledProcessError: Command '['sox', 'C:\\Users\\ahe42\\AppData\\Local\\Temp\\tmpu1vxf9m6.wav', 'C:\\Users\\ahe42\\AppData\\Local\\Temp\\tmpnoc6muw_.wav', '-q', 'compand', '0.1,0.3', '-90,-90,-70,-58,-55,-43,-31,-31,-21,-21,0,-20', '0', '0', '0.1']' returned non-zero exit status 1.